# report

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import os
import copy
import holidays
import datetime
from contract_select import contract_select
from hedge_calc import hedge_calc
from greek_calc import greek_calc, pnl_daily_calc
from position_adj_cost import position_adj_cost, initial_position_cost

# Set the first trading date

In [30]:
moneyness = 'out'
main_type = 'P'
hedge_type = 'P'
main_minDistance = 0.08
hedge_minDistance = 0.06
starting_date = '2018-12-10'

main_original = contract_select(select_type='main', date= starting_date, max_spread_pct = 0.05, spy_spx='spx', contract_type = main_type, minAsk=1.0, minDistance = main_minDistance, minDTE=15, moneyness=moneyness, num = 2)
hedge_original = contract_select(select_type='hedge', date= starting_date, max_spread_pct = 0.05, spy_spx='spx', contract_type = hedge_type, minAsk=0.2, minDistance = hedge_minDistance, minDTE=15, moneyness=moneyness, num = 3)
main_original.update(hedge_original)

initial_position = hedge_calc(main_original)
initial_spread_cost = initial_position_cost(initial_position)

# Wind up position every friday and day before any holiday

In [42]:
greek_pnl_daily = pd.DataFrame()
greek_pnl_daily['daily_pnl'] = 0
greek_pnl_daily['adjust_cost'] = 0

position = copy.deepcopy(initial_position)
position['date'] = starting_date
daily = []

for i in range(30):
    greek_pnl_daily = greek_pnl_daily.append(greek_calc(position))
    position = pnl_daily_calc(position)
    date = position['date']
    greek_pnl_daily['daily_pnl'][-1] =  position['daily_rtn']
    daily.append(position)
           
    temp = copy.deepcopy(position)
    # drop/wind up certain main contracts that meet some criteria
    if len(temp['main']) != 0:
        if main_type == 'P': 
            #drop ITM contracts
            temp['main'] = temp['main'][temp['main']['[UNDERLYING_LAST]'] > temp['main']['[STRIKE]']]
            #minimal ask needs to be above 0.5
            # temp['main'] = temp['main'][temp['main']['[P_ASK]'] > 0.5]
        if main_type == 'C': 
            #drop ITM contracts
            temp['main'] = temp['main'][temp['main']['[UNDERLYING_LAST]'] < temp['main']['[STRIKE]']]
            #minimal ask needs to be above 0.5
            # temp['main'] = temp['main'][temp['main']['[C_ASK]'] > 0.5]
        #minimum DTE need to be at least 5 days
        temp['main'] = temp['main'][temp['main']['[DTE]'] > 5]
        #minimum distance need to be half the main_minDistance
        temp['main'] = temp['main'][temp['main']['[STRIKE_DISTANCE_PCT]'] >= main_minDistance/2]
    #select new contracts if num of contracts is less than 2
    if len(temp['main']) < 2:
        new_contract = contract_select(select_type='main', date = date, max_spread_pct = 0.05, spy_spx='spx', contract_type = main_type, minAsk = 1.0, minDistance = main_minDistance, minDTE = 15, moneyness=moneyness, num=2-len(temp['main']))
        temp['main'] = temp['main'].append(new_contract['main'])
        temp['main']['WEIGHT'] = -1/len(temp['main'])

    # drop/wind up certain hedge contracts that meet some criteria
    if len(temp['hedge']) != 0:
        if hedge_type == 'P': 
            #drop ITM contracts
            temp['hedge'] = temp['hedge'][temp['hedge']['[UNDERLYING_LAST]'] > temp['hedge']['[STRIKE]']]
        if hedge_type == 'C': 
            #drop ITM contracts
            temp['hedge'] = temp['hedge'][temp['hedge']['[UNDERLYING_LAST]'] < temp['hedge']['[STRIKE]']]
        #minimum DTE need to be at least 5 days
        temp['hedge'] = temp['hedge'][temp['hedge']['[DTE]'] > 5]
        #minimum distance need to be half the main_minDistance
        temp['hedge'] = temp['hedge'][temp['hedge']['[STRIKE_DISTANCE_PCT]'] >= hedge_minDistance/2]
    #select new contracts if num of contracts is less than 3
    if len(temp['hedge']) < 3:
            new_contract = contract_select(select_type = 'hedge', date = date, max_spread_pct = 1, spy_spx = 'spx', contract_type = hedge_type, minAsk = 0.2, minDistance = hedge_minDistance, minDTE = 15, moneyness=moneyness, num = 3 - len(temp['hedge']))
            temp['hedge'] = temp['hedge'].append(new_contract['hedge'])
    
    temp = hedge_calc(temp)                
    greek_pnl_daily['adjust_cost'][-1] = position_adj_cost(position, temp)['adjust_cost']
    position=copy.deepcopy(temp)

# Ignore any holiday and friday

# Wind up position every friday

In [44]:
greek_pnl_daily.to_clipboard()

In [ ]:
df = daily[-1]['main']
df

In [43]:
greek_pnl_daily

,daily_pnl,adjust_cost,DELTA,GAMMA,VEGA,THETA
2018-12-10,0.006287,-2.719766e-05,0.0,-0.000657,-0.636530,0.499719
2018-12-11,0.004261,-1.228739e-04,0.0,-0.000661,-0.572170,0.463749
2018-12-12,-0.000015,-4.484119e-04,0.0,-0.000563,-0.439580,0.367898
2018-12-13,0.014238,-3.063188e-05,0.0,-0.000498,-0.367274,0.323286
2018-12-14,-0.019418,-1.663686e-04,0.0,-0.000893,-0.514283,0.463413
2018-12-17,0.002259,-4.442030e-05,0.0,-0.001568,-0.526244,0.936807
2018-12-18,0.005649,-3.057320e-04,0.0,-0.001625,-0.412034,0.991013
2018-12-19,-0.019284,-1.990582e-04,0.0,-0.000635,-0.971643,0.449561
2018-12-20,0.000557,-2.582714e-05,0.0,-0.000774,-1.251926,0.676149
2018-12-21,-0.026073,-5.828866e-04,0.0,-0.001082,-1.545342,0.812121


In [ ]:
main_original

In [ ]:
d['main']['WEIGHT']

In [ ]:
nyse_holidays = holidays.financial_holidays('NYSE')

In [ ]:
'01/01/2000' in nyse_holidays

In [4]:
datetime.date.weekday(datetime.date(2018,1,2))

1

In [8]:
datetime.date.fromisoformat(position['date'])

datetime.date(2018, 12, 12)

In [ ]:
position